This notebook retrieves the mining districts found in the mineral deposits [website](https://mrdata.usgs.gov/deposit/). It first pulls the WGS layers found in the API endpoint and retrieves the polygon layer which has the mining districts of interest.

In [64]:
import geopandas as gpd
import requests
import fiona
import os
from fiona import BytesCollection
from shapely.geometry import Polygon
from wbmaker import WikibaseConnection
from dotenv import load_dotenv
from arcgis.gis import GIS

load_dotenv()

fiona.drvsupport.supported_drivers['WFS'] = 'r'

In [65]:
def wfs2gp_df(layer_name, url, bbox=None, wfs_version="2.0.0", outputFormat='application/gml+xml; version=3.2'):
    params = dict(service='WFS', version=wfs_version,request='GetCapabilities', typeName=layer_name, outputFormat=outputFormat)
    with BytesCollection(requests.get(url,params=params).content) as f:
        df = gpd.GeoDataFrame.from_features(f, crs='EPSG:4326')
    return df

mining_districts = wfs2gp_df('polygons', 'https://mrdata.usgs.gov/services/wfs/deposit',None, '1.1.0', 'json')

In [66]:
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks
0,"POLYGON ((46.74615 -87.88469, 46.74619 -87.884...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
1,"POLYGON ((47.82946 -91.67692, 47.82990 -91.679...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
2,"POLYGON ((46.74872 -87.89662, 46.74872 -87.896...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
3,"POLYGON ((47.81905 -91.72215, 47.81936 -91.721...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
4,"POLYGON ((47.71916 -91.81861, 47.71935 -91.820...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN


In [67]:
def reverse_coords(geom: Polygon) -> Polygon:
    coords = geom.exterior.coords.xy
    reverse = tuple(zip(coords[1], coords[0]))
    return Polygon(reverse)

In [68]:
mining_districts['geometry'] = [reverse_coords(x) for x in mining_districts['geometry']]
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
2,"POLYGON ((-87.89662 46.74872, -87.89657 46.748...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
3,"POLYGON ((-91.72215 47.81905, -91.72119 47.819...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
4,"POLYGON ((-91.81861 47.71916, -91.82029 47.719...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN


In [148]:
(mining_districts
# .loc[mining_districts['ftr_name']
# .str
# .contains('Eagle') ]
# .groupby(['site_id', 'geometry', 'ftr_name','doi']).apply(lambda x : x)
.drop_duplicates(subset=['site_id'])
)

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid,label,aliases
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176),Eagle East Mining District,Eagle East
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981),Spruce Road Mining District,Spruce Road
5,"POLYGON ((-91.95769 47.62485, -91.95477 47.625...",polygons.24,MN00002,Mo00746,NorthMet,2018-04-16,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.97153846066311 47.61665047292834),NorthMet Mining District,NorthMet
7,"POLYGON ((-90.11476 37.61021, -90.11857 37.528...",polygons.27,MO00001,Mr00036,Fredericktown District,2018-06-07,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-90.2061544546558 37.57042461027146),Fredericktown District Mining District,Fredericktown District
9,"POLYGON ((-76.41196 40.26707, -76.41213 40.267...",polygons.41,PA00001,Mo00749,Cornwall,2018-04-18,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-76.4122036035501 40.26710794585181),Cornwall Mining District,Cornwall
...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,"POLYGON ((-118.70941 34.85643, -118.71769 34.8...",polygons.241,CA00066,Mr00143,Gorman District,2018-04-13,10.5066/P97JYNJL,NaN,NaN,NaN,POINT (-118.74650803386271 34.842911735946814),Gorman District Mining District,Gorman District
665,"POLYGON ((-118.74192 34.83684, -118.74189 34.8...",polygons.242,CA00067,Mo00725,"Meeke, West gossan",2018-04-13,10.5066/P97JYNJL,NaN,NaN,NaN,POINT (-118.74195102441725 34.83680623844576),"Meeke, West gossan Mining District","Meeke, West gossan"
666,"POLYGON ((-118.72938 34.84020, -118.72937 34.8...",polygons.243,CA00068,Mo00727,Upper Butler,2018-04-13,10.5066/P97JYNJL,NaN,NaN,NaN,POINT (-118.72901749491731 34.8400941218302),Upper Butler Mining District,Upper Butler
668,"POLYGON ((-113.90865 34.92930, -113.91214 34.9...",polygons.268,AZ00002,Mr00131,Boriana Mineral District,2020-05-01,10.5066/P9XA8MJ4,7.31962822544,1808.71952485,NaN,POINT (-113.9014722791027 34.94781586205533),Boriana Mineral District Mining District,Boriana Mineral District


In [70]:
gis = GIS()
counties = gis.content.get('14c5450526a8430298b2fa74da12c2f4')
counties


/tmp/geokb/geokbenv/lib/python3.8/site-packages/arcgis/gis/_impl/_con/_connection.py:296: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  max_retries=Retry(
/tmp/geokb/geokbenv/lib/python3.8/site-packages/arcgis/gis/__init__.py:11174: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  numViews = locale.format("%d", self.numViews, grouping=True)


<Item title:"USA Census Counties" type:Feature Layer Collection owner:esri_dm>

In [71]:
layer = counties.layers[0]
q = layer.query(where="1=1", out_fields=['*'], as_df=True, out_sr=4326)

In [72]:
q.head()

,COUNTY_FIPS,FIPS,NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,SQMI,STATE_ABBR,STATE_FIPS,STATE_NAME,Shape__Area,Shape__Length
0,001,01001,Autauga County,1,58805,97.3,"{""rings"": [[[-86.413120727, 32.7073921370001],...",604.37,AL,01,Alabama,0.150256,2.066033
1,003,01003,Baldwin County,2,231767,141.9,"{""rings"": [[[-87.5649079999999, 30.281622], [-...",1633.14,AL,01,Alabama,0.398404,9.305629
2,005,01005,Barbour County,3,25223,27.9,"{""rings"": [[[-85.257838372, 32.147937056], [-8...",904.52,AL,01,Alabama,0.223270,2.695260
3,007,01007,Bibb County,4,22293,35.6,"{""rings"": [[[-87.06574294, 33.2469132270001], ...",626.17,AL,01,Alabama,0.156473,1.887519
4,009,01009,Blount County,5,59134,90.9,"{""rings"": [[[-86.453024823, 34.259323463], [-8...",650.63,AL,01,Alabama,0.164405,2.423466


In [95]:
MN_counties = q.query('STATE_NAME == "Minnesota"')

In [74]:
from shapely import wkt
mining_districts['centroid'] = [wkt.loads(str(x)).centroid for x in mining_districts['geometry']]
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176)
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981)
2,"POLYGON ((-87.89662 46.74872, -87.89657 46.748...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.89599174151172 46.74811460369112)
3,"POLYGON ((-91.72215 47.81905, -91.72119 47.819...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.71992813450217 47.818018850826434)
4,"POLYGON ((-91.81861 47.71916, -91.82029 47.719...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.80634116774306 47.733082499865596)


In [160]:
def fix_label(label: str) -> str:
    if 'Mining District' not in label:
        return label  + ' Mining District'
    if 'District' in label and not 'Mining District' in label:
        return label.replace('District','Mining District')
    return label

labels = [ fix_label(x) for x in mining_districts['ftr_name'] ]
aliases = [ x if 'mining district' not in x.lower() else '' for x in mining_districts['ftr_name'] ]

In [161]:
mining_districts['label'] = labels
mining_districts['aliases'] = aliases

In [162]:
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid,label,aliases
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176),Eagle East Mining District,Eagle East
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981),Spruce Road Mining District,Spruce Road
2,"POLYGON ((-87.89662 46.74872, -87.89657 46.748...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.89599174151172 46.74811460369112),Eagle Mining District,Eagle
3,"POLYGON ((-91.72215 47.81905, -91.72119 47.819...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.71992813450217 47.818018850826434),Maturi Mining District,Maturi
4,"POLYGON ((-91.81861 47.71916, -91.82029 47.719...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.80634116774306 47.733082499865596),Birch Lake Mining District,Birch Lake


In [163]:
mining_districts.loc[mining_districts['ftr_name'] == 'Blue Wing Mining District']

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid,label,aliases
316,"POLYGON ((-113.62359 44.59117, -113.58776 44.5...",polygons.282,ID00004,Mr00153,Blue Wing Mining District,2020-04-21,10.5066/P9XA8MJ4,72.0497945628,17803.8919701,NaN,POINT (-113.66513646127024 44.54044553228987),Blue Wing Mining District,
434,"POLYGON ((-113.62359 44.59117, -113.58776 44.5...",polygons.640,ID00004,Mr00153,Blue Wing Mining District,2020-04-21,10.5066/P9XA8MJ4,72.0497945628,17803.8919701,NaN,POINT (-113.66513646127024 44.54044553228987),Blue Wing Mining District,


In [137]:
[x  for x in mining_districts['ftr_name'] if 'mining district' in x.lower()]

['Blue Wing Mining District',
 'Butte Mining District',
 'Lost Creek Mining District',
 'Bishop Mining District',
 'Chidago Mining District',
 'Bayhorse Mining District',
 'Yellow Pine Mining District',
 'Grimes Pass Mining District',
 'Wise River Mining District',
 'Potosi Mining District',
 'White Pine Mining District',
 'Pilot Mountains Mining District',
 'Mill City Mining District',
 'Nightingale Mining District',
 'Delano Mining District',
 'Golconda Mining District',
 'Lodi Mining District',
 'Tem Piute Mining District',
 'Shoshone Mining District',
 'Silver Star Mining District',
 'Leonard Mining District',
 'Cedar Canyon Mining District',
 'Tutsagubet Mining District',
 'Silver Star Mining District',
 'Leonard Mining District',
 'Blue Wing Mining District',
 'Butte Mining District',
 'Lost Creek Mining District',
 'Bishop Mining District',
 'Chidago Mining District',
 'Bayhorse Mining District',
 'Yellow Pine Mining District',
 'Grimes Pass Mining District',
 'Wise River Mining

In [97]:
MN_counties[MN_counties['NAME'].str.contains('Louis')]

,COUNTY_FIPS,FIPS,NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,SQMI,STATE_ABBR,STATE_FIPS,STATE_NAME,Shape__Area,Shape__Length
1383,137,27137,St. Louis County,1384,200231,29.7,"{""rings"": [[[-92.283059668, 46.658813604], [-9...",6734.11,MN,27,Minnesota,2.08628,7.674399


In [75]:
stl = MN_counties[MN_counties['NAME'].str.contains('Louis')]['SHAPE'].values[0]

In [98]:
birch_lake = mining_districts.iloc[4]
birch_lake

geometry      POLYGON ((-91.818614 47.71916, -91.820289 47.7...
gml_id                                              polygons.23
site_id                                                 MN00003
ftr_id                                                  Mo00719
ftr_name                                             Birch Lake
last_updt                                            2018-04-10
doi                                            10.5066/P9V74HIU
area_sqkm                                                   NaN
area_acres                                                  NaN
remarks                                                     NaN
centroid          POINT (-91.80634116774306 47.733082499865596)
Name: 4, dtype: object

In [105]:
# intended to write a function that matches the polygons and retrieves state/county information from match
#TODO
# contained = False
# for i in range(len(ls)):
#     shp = ls.iloc[i]['SHAPE']
#     s2_shp = MultiPolygon([s2['geometry']])
#     # print(type(shp), type(s2_shp))
#     contained = s2_shp.touches(shp.as_shapely)
#     if contained:
#         break
# print(contained)

False


In [99]:
str(birch_lake['geometry'])

'POLYGON ((-91.818614 47.71916, -91.820289 47.71935, -91.820442 47.720188, -91.819604 47.721178, -91.81831 47.721863, -91.815797 47.724071, -91.814426 47.725366, -91.812066 47.725556, -91.810619 47.72685, -91.810924 47.728564, -91.812866 47.732142, -91.812409 47.734008, -91.813018 47.736483, -91.812142 47.737739, -91.812028 47.739224, -91.809516 47.741927, -91.808602 47.743183, -91.806508 47.744744, -91.803653 47.745315, -91.801064 47.746914, -91.79977 47.747067, -91.797714 47.746343, -91.796001 47.745506, -91.795696 47.74463, -91.797181 47.744287, -91.800759 47.744325, -91.802853 47.744173, -91.802511 47.741965, -91.798665 47.741242, -91.798133 47.74048, -91.798742 47.739909, -91.798742 47.73949, -91.795429 47.736749, -91.795582 47.736026, -91.79661 47.735759, -91.798056 47.734655, -91.799655 47.733094, -91.800341 47.731381, -91.799731 47.73062, -91.798399 47.729668, -91.797333 47.728906, -91.79798 47.728221, -91.799389 47.727726, -91.800302 47.727079, -91.80133 47.726089, -91.80213 4

In [101]:
type(stl), type(stl.as_shapely), type(birch_lake['geometry'])

(arcgis.geometry._types.Polygon,
 shapely.geometry.multipolygon.MultiPolygon,
 shapely.geometry.polygon.Polygon)

In [106]:
MultiPolygon([birch_lake['geometry']]).overlaps(stl.as_shapely)

False

In [ ]:
name = 'GEOKB_CLOUD'
geokb = WikibaseConnection(name)

In [19]:
def item_search(label: str, instance_of: str, bot_name: str):
  sparql_endpoint = os.environ[f'WB_SPARQL_{bot_name}']
  query = f'''PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>
  PREFIX wd:  <https://geokb.wikibase.cloud/entity/>
  SELECT ?item
  WHERE {{
    ?item rdfs:label ?label ;
       wdt:P1 wd:{instance_of} .
    FILTER CONTAINS( LCASE(?label), "{label.lower()}") .

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en" . }}
  }}
  '''

  params = {
      'query': query,
      'format': 'json'
  }

  res = requests.get(sparql_endpoint, params=params, timeout=100)
  json_res =res.json()
  item_result = (json_res['results']['bindings'][0]['item']['value']
                  if 'results' in json_res
                  and len(json_res['results']['bindings']) > 0
                  and 'item' in json_res['results']['bindings'][0]
                  else None)
  return item_result.split('/')[-1] if item_result is not None else None

In [20]:
districts_ref = geokb.datatypes.URL(
    prop_nr=geokb.prop_lookup['reference URL'],
    value='https://mrdata.usgs.gov/deposit/'
)

In [33]:
len(mining_districts)

713

In [ ]:
def add_new_item(cols:dict, instance_of_val: str):
    keep = geokb.action_if_exists.KEEP
    aor = geokb.action_if_exists.APPEND_OR_REPLACE


    item = geokb.wbi.item.new()
    references = geokb.models.References()
    updt_ref = geokb.datatypes.Time(
        prop_nr=geokb.prop_lookup['last update'],
        time=cols['last_updt']+'T00:00:00Z'
    )
    references.add(districts_ref)
    references.add(updt_ref)
    item.labels.set('en', cols['ftr_name'], action_if_exists=keep)
    item.descriptions.set('en', '')
    item.claims.add(
        geokb.datatypes.Item(
                prop_nr=geokb.prop_lookup["instance of"],
                value=instance_of_val,
                references=references,
                action_if_exists=keep
        )
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["Feature ID"],
            value=cols['ftr_id'],
            references=references,
            action_if_exists=keep
        )
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["Site ID"],
            value=cols['site_id'],
            references=references,
            action_if_exists=keep
        )
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["DOI"],
            value=cols['doi'],
            references=references,
            action_if_exists=keep
        )
    )
    response = item.write(
        summary="Added mining districts"
    )
    print(cols['ftr_name'], response.id)

In [34]:
instance_of_val = 'Q55213'
for i in range(len(mining_districts)):
    fields = dict(mining_districts.iloc[i])
    qid = item_search(fields['ftr_name'], instance_of_val, name)
    if qid is None:
        add_new_item(fields, instance_of_val)
    else:
        print(f'{fields["ftr_name"]} found in the GeoKB, ID: {qid}')

Eagle East found in the GeoKB, ID: Q157855
Spruce Road Q157856
Eagle found in the GeoKB, ID: Q55216
Maturi Q157857
Birch Lake Q157858
NorthMet Q157859
Maturi Southwest Q157860
Fredericktown District Q157861
Fredericktown District Q157862
Cornwall Q157863
Ducktown District Q157864
Regional Smackover brine Q157865
Smackover-Lanxess, West Unit Q157866
Smackover-Lanxess, Central Unit Q157867
Smackover-Lanxess, South Unit Q157868
Smackover-Tetra, South Resource Area Q157869
Smackover-Tetra, North Resource Area Q157870
Kings Mountain District Q157871
Piedmont, Core deposit Q157872
Bessemer City, Bessemer City deposit Q157873
Kings Mountain, Foote deposit Q157874
Trail Ridge, Highland Zone Q157875
Magnet Cove District Q157876
Boulogne, Boulougne Deposit Q157877
Green Cove Springs, Green Cove Springs Deposit Q157878
Trail Ridge, Trail Ridge Zone Q157879
Trail Ridge, Maxville Zone Q157880
Jacksonville, Jacksonville Deposit Q157881
Little Talbot Island, Little Talbot Island Prospect Q157882
Folk

ValueError: Time value (+2018-3-16T00:00:00Z) must be a string in the following format: '+%Y-%m-%dT00:00:00Z'.